In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from sklearn.datasets import make_blobs
import random
import json
import time
from tabulate import tabulate

from kdtree import *
from helper import *

In [2]:
savefile = "mst_data.json"
to_plot = False
no_points = 1000
no_centres = 10

In [3]:
X, Y = make_blobs(n_samples=no_points, centers=no_centres, random_state=42)
points = [(x, y) for x, y in X]

In [4]:
cordmap = {point: i for i, point in enumerate(points)}

In [5]:
def dcran(cordmap):
    tree = KDTree()
    tree.root = tree.build(points)
    G = nx.Graph()
    for coord, index in cordmap.items():
        G.add_node(index, pos=coord)

    k = 2
    connected_components = 1
    while True:
        maxdis = min(len(cordmap) - 1, 2**k)
        for pointi, i in cordmap.items():
            pointj = ith_nearest_neighbor(tree, pointi, k)
            dis = euclidean_distance(pointi, pointj)
            G.add_edge(i, cordmap[pointj], weight=dis)
            # print(k ,  i , cordmap[pointj] , dis )
            randis = random.randint(k + 1, maxdis)
            # randis = (cusrandis(k + 1 , maxdis))
            # print(randis , k)
            pointj = ith_nearest_neighbor(tree, pointi, randis)
            dis = euclidean_distance(pointi, pointj)
            G.add_edge(i, cordmap[pointj], weight=dis)
            # print(k ,  i , cordmap[pointj] , dis , randis)
        ccount = count_clusters(G)
        print(k, ccount)
        k += 1
        if ccount == 1:
            break

    return G

In [6]:
dcran_start_time = time.time()
kc = dcran(cordmap)

2 23
3 6


4 6
5 6
6 5
7 1


In [7]:
graphify(kc, to_plot)

In [8]:
G = kc
num_nodes = G.number_of_nodes()
conedge = num_edges = G.number_of_edges()

# Calculate the sum of all edge weights
total_weight = sum(data["weight"] for u, v, data in G.edges(data=True))

print(f"Total number of nodes: {num_nodes}")
print(f"Total number of edges: {num_edges}")
print(f"Total sum of edge weights: {total_weight}")

Total number of nodes: 1000
Total number of edges: 7168
Total sum of edge weights: 5944.512258975906


In [9]:
mst = nx.minimum_spanning_tree(G, algorithm="kruskal", weight="weight")
graphify(mst, to_plot)

print("Edges in the MST:", list(mst.edges(data=True)), sep="\n")

Edges in the MST:
[(0, 750, {'weight': 0.4046577497950422}), (1, 547, {'weight': 0.21166735838710274}), (1, 424, {'weight': 0.2529097795779661}), (2, 30, {'weight': 0.1972470807515501}), (2, 651, {'weight': 0.3434624844540697}), (3, 252, {'weight': 0.17432208623467957}), (3, 542, {'weight': 0.23428484400519134}), (4, 119, {'weight': 0.09189273775367071}), (4, 773, {'weight': 0.1308454443999734}), (5, 30, {'weight': 0.1140988164084683}), (5, 255, {'weight': 0.22767417863369988}), (6, 208, {'weight': 0.12278354115431817}), (6, 927, {'weight': 0.33043740146298045}), (7, 102, {'weight': 0.183086296356968}), (7, 879, {'weight': 0.22974248386869786}), (8, 578, {'weight': 0.10073693348992833}), (8, 802, {'weight': 0.3811571726174638}), (8, 443, {'weight': 0.3967609686541967}), (9, 561, {'weight': 0.4717239963273117}), (9, 217, {'weight': 0.6628179017538243}), (10, 721, {'weight': 0.20705477038970255}), (10, 988, {'weight': 0.2509370474079433}), (11, 694, {'weight': 0.17250835299513737}), (11,

In [10]:
G = mst
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()


foundw = total_weight = sum(data["weight"] for u, v, data in G.edges(data=True))

print(f"Total number of nodes: {num_nodes}")
print(f"Total number of edges: {num_edges}")
print(f"Total sum of edge weights: {total_weight}")
dcran_end_time = time.time()
dcran_elapsed_time = dcran_end_time - dcran_start_time

Total number of nodes: 1000
Total number of edges: 999
Total sum of edge weights: 263.7351945892295


In [11]:
stmst_start_time = time.time()
G = nx.Graph()

for pointi, i in cordmap.items():
    G.add_node(i, pos=pointi)
    for pointj, j in cordmap.items():
        if i != j:
            dis = euclidean_distance(pointi, pointj)
            G.add_edge(i, j, weight=dis)

In [12]:
mst = nx.minimum_spanning_tree(G, algorithm="kruskal", weight="weight")
graphify(mst, to_plot)

In [13]:
G = mst
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

# Calculate the sum of all edge weights
realw = total_weight = sum(data["weight"] for u, v, data in G.edges(data=True))

print(f"Total number of nodes: {num_nodes}")
print(f"Total number of edges: {num_edges}")
print(f"Total sum of edge weights: {total_weight}")
stmst_end_time = time.time()
stmst_elapsed_time = stmst_end_time - stmst_start_time

Total number of nodes: 1000
Total number of edges: 999
Total sum of edge weights: 258.819134723451


In [14]:
percentage_error = ((foundw - realw) / realw) * 100

# Format the output for readability
formatted_output = f"Real Weight: {realw}  Found Weight: {foundw}  no of Edge: {conedge} Percentage Error: {percentage_error:.2f}%"
formatted_output

'Real Weight: 258.819134723451  Found Weight: 263.7351945892295  no of Edge: 7168 Percentage Error: 1.90%'

In [15]:
with open(savefile, "r") as f:
    loaded_data = json.load(f)
print(loaded_data)
currres = []
speedup = round((stmst_elapsed_time  / dcran_elapsed_time) , 2)
loaded_data.append(
    [
        no_points,
        no_centres,
        foundw,
        realw,
        conedge,
        round(100 - percentage_error , 3),
        round(dcran_elapsed_time , 2),
        round(stmst_elapsed_time ,2),
        speedup
    ]
)

# Save the updated dictionary back to the JSON file
with open(savefile, "w") as f:
    json.dump(loaded_data, f)

[[1000, 50, 385.5403470823181, 383.766271717251, 2376, 99.54, 0.18, 7.97, 44.01], [1000, 5, 206.6799210789751, 204.64707028626765, 8418, 99.01, 1.35, 2.81, 2.09], [10000, 5, 674.8216570832636, 673.1287511582893, 125297, 99.749, 80.65, 640.4, 7.94]]


In [16]:
with open("mst_data.json", "r") as f:
    loaded_data = json.load(f)
print(loaded_data)

[[1000, 50, 385.5403470823181, 383.766271717251, 2376, 99.54, 0.18, 7.97, 44.01], [1000, 5, 206.6799210789751, 204.64707028626765, 8418, 99.01, 1.35, 2.81, 2.09], [10000, 5, 674.8216570832636, 673.1287511582893, 125297, 99.749, 80.65, 640.4, 7.94], [1000, 10, 263.7351945892295, 258.819134723451, 7168, 98.101, 1.15, 3.94, 3.42]]


In [17]:
headers = [
    "Points",
    "Centres",
    "DCRAN Wt",
    "kruskal Wt",
    "Edge count",
    "Acc(%)",
    "DCRAN Time (s)",
    "STMST Time (s)",
    "Speedup"
]

# Format the data as a table using tabulate
table_str = tabulate(loaded_data, headers, tablefmt="pipe", floatfmt=(".0f", ".0f", ".1f", ".1f", ".0f", ".1f", ".2f", ".2f", ".2f"))
print(table_str)




|   Points |   Centres |   DCRAN Wt |   kruskal Wt |   Edge count |   Acc(%) |   DCRAN Time (s) |   STMST Time (s) |   Speedup |
|---------:|----------:|-----------:|-------------:|-------------:|---------:|-----------------:|-----------------:|----------:|
|     1000 |        50 |      385.5 |        383.8 |         2376 |     99.5 |             0.18 |             7.97 |     44.01 |
|     1000 |         5 |      206.7 |        204.6 |         8418 |     99.0 |             1.35 |             2.81 |      2.09 |
|    10000 |         5 |      674.8 |        673.1 |       125297 |     99.7 |            80.65 |           640.40 |      7.94 |
|     1000 |        10 |      263.7 |        258.8 |         7168 |     98.1 |             1.15 |             3.94 |      3.42 |


In [18]:
print("Line 1\nLine 2")


Line 1
Line 2
